# Quantum Simulated Annealing optimization using Monte Carlo methods

In [36]:
# Imports
%matplotlib inline
import numpy as np
import math
from random import randint
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['interactive'] == True
import time
from scipy.optimize import minimize
from qiskit_aer.aerprovider import AerSimulator
from qiskit_aer import Aer, aerprovider
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import RealAmplitudes
from qiskit.circuit.library import HGate, ZGate
from qiskit.circuit import ClassicalRegister, QuantumRegister
from qiskit.primitives import StatevectorEstimator as Estimator
from qiskit.primitives import StatevectorSampler as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import *

## Helper functions

In [37]:
'''
Reversely find a parameter.

Input y is desired prepared quantum state. (L2 Normalized |y|^2 = 1)
Input optimization_method is function used to optimize the parameters.
Find parameters of variational quantum circuit "RealAmplitude" that can prepare y.
'''
def find_parameter(y, optimization_method: callable):
    n = int(math.log2(len(y))) 
    ansatz = RealAmplitudes(n, reps=1) 
    parameters = np.ones(ansatz.num_parameters)  
    simulator = AerSimulator(method='statevector') 
    meas = ClassicalRegister(1, "meas") 
    qreg = QuantumRegister(2 * n + 1) 
    c = QuantumCircuit(qreg, meas) 

    # Prepare the circuit for the swap test
    a = [1]
    b = [n + 1]
    for i in range(2, n + 1):
        a.append(i)
        b.append(n + i)

    final = c.compose(ansatz, a) 
    final.initialize(y, b) 
    final.save_statevector(label="ans") 
    final.h(0) 

    # Perform the swap test
    for i in range(1, n + 1):
        final.cswap(0, i, n + i) 
    final.h(0) 
    final.measure([0], meas) 

    # Transpile and run the circuit
    circ = transpile(final, simulator) 
    circfinal = circ.assign_parameters(parameters) 
    results = simulator.run(circfinal).result() 
    init_counts = results.get_counts()
    print(f"Counts before optimization: {init_counts}") 
    
    # Visualize results of the original implementation
    # visualize_results(init_counts, "Before Optimization")

    # Optimization
    Energy = E(parameters, circ, simulator) 
    anneal = optimization_method(100, parameters, circ, simulator) 
    print(f"Energy: {Energy}") 

    # Update circuit with optimized parameters and run again
    circfinal = circ.assign_parameters(anneal)
    results = simulator.run(circfinal).result()
    optimized_counts = results.get_counts() 
    

    # Calculate and print results
    b = optimized_counts.get('1', 0) # Number of times qc measured '1'
    s = (1 - (2 / 1024) * b) # Success value
    # circfinal.draw(output="mpl")
    #matplotlib.pyplot.show() 
    print(f"After optimization: {str(results.get_counts())}") 
    print(f"Success value: {s}")

    # Visualize results of the Monte Carlo optimization
    # visualize_results(optimized_counts, "After Optimization")

    # Calculate and print partial traces
    temp = partial_trace(results.data(0)['ans'], [0, 3, 4])
    partial = np.diagonal(temp)
    temp = partial_trace(results.data(0)['ans'], [0, 1, 2])
    partial2 = np.diagonal(temp)
    norm = np.linalg.norm(partial - partial2) 
    print("Norm of the difference between partial traces:", norm)

    return parameters 

# Function to calculate cost based on parameters
def cost_func(params, ansatz, simulator):
    circfinal = ansatz.assign_parameters(params) 
    results = simulator.run(circfinal, shots=1024).result() 
    counts = results.get_counts() 
    b = counts.get('1', 0) 
    s = -1 * (1 - ((2 / 1024) * b)) 
    return s 

# Function to calculate energy based on parameters
def E(params, ansatz, simulator):
    circfinal = ansatz.assign_parameters(params) 
    results = simulator.run(circfinal, shots=20).result() 
    temp = partial_trace(results.data(0)['ans'], [0, 3, 4])
    partial = np.diagonal(temp) 
    temp = partial_trace(results.data(0)['ans'], [0, 1, 2])
    partial2 = np.diagonal(temp) 
    norm = np.linalg.norm(partial - partial2) 
    return norm 

# Function to visualize the results of the quantum circuit
def visualize_results(counts, title):
    plt.bar(counts.keys(), counts.values())
    plt.title(title)
    plt.xlabel('Measurement Outcome')
    plt.ylabel('Counts')
    plt.show()


## Monte Carlo Optimization Functions

In [44]:
# Simulated Annealing optimization
def simulated_annealing(runs, params, ansatz, simulator):
    B = E(params, ansatz, simulator) 
    prev_E = B

    # Temperature function for simulated annealing
    def T(t):
        c = 0.02
        a = 0.01
        temperature = c / (a + math.log(t)) 
        return temperature

    # Main loop for simulated annealing
    for t in range(1, runs):
        delta = np.random.normal(0, .1, 4) 
        params_new = params + delta 
        E_new = E(params_new, ansatz, simulator) 
        delta_E = E_new - prev_E 

        if delta_E <= 0:
            params = params_new
            prev_E = E_new
        else:
            h = math.pow(math.e, -1 * delta_E / T(t)) 
            U = np.random.normal(.5, .5, 1) 
            if U < h:
                params = params_new
                prev_E = E_new

    return params 

# Global Best Particle Swarm optimization
def gbest_pso(runs, params, ansatz, simulator):
    num_particles = 40  # Number of particles in the swarm
    dimensions = len(params)  # Number of parameters to optimize

    particles = np.random.rand(num_particles, dimensions) * 2 - 1  # Random positions in range [-1, 1]
    velocities = np.random.rand(num_particles, dimensions) * 0.1  # Small random velocities
    personal_best_positions = np.copy(particles)  # Initial best positions for each particle
    personal_best_scores = [cost_func(p, ansatz, simulator) for p in particles]  # Initial best scores for each particle

    global_best_position = personal_best_positions[np.argmin(personal_best_scores)]  # Best position from all particles

    for _ in range(runs):
        # Update personal best and global best
        for i in range(num_particles):
            score = cost_func(particles[i], ansatz, simulator)  # New score

            # Update personal best
            if score < personal_best_scores[i]:
                personal_best_scores[i] = score
                personal_best_positions[i] = particles[i]

            # Update global best
            if score < cost_func(global_best_position, ansatz, simulator):
                global_best_position = particles[i]

        # Update velocities and positions
        for i in range(num_particles - 1):
            c1 = 0.4  # Personal best weight
            c2 = 0.4  # Global best weight
            r1 = np.random.rand(dimensions)
            r2 = np.random.rand(dimensions)

            # Update velocity
            velocities[i+1] = (velocities[i] + 
                            c1 * r1 * (personal_best_positions[i] - particles[i]) + 
                            c2 * r2 * (global_best_position - particles[i]))

            # Update position
            particles[i+1] = particles[i] + velocities[i+1]

    return global_best_position

# Differential Evolution optimization
def diff_evolution(runs, params, ansatz, simulator):
    # To implement
    return params

# Stochastic Hill Climbing optimization
def stochastic_hill_climbing(runs, params, ansatz, simulator):
    # To implement
    return params

# Compare the results

In [46]:
# Main execution block
randvect = [randint(0, 100) for p in range(0, 4)] 
norm = np.linalg.norm(randvect) 
randvect = randvect / norm 
print(f"Random vector: {randvect}\n")

# Timing for Simulated Annealing
start_time = time.time()
original_parameters_sa = find_parameter(randvect, simulated_annealing)
sa_time = time.time() - start_time
print(f"Simulated Annealing Time: {sa_time:.4f} seconds\n")

# Timing for Particle Swarm Optimization
start_time = time.time()
original_parameters_pso = find_parameter(randvect, gbest_pso)
pso_time = time.time() - start_time
print(f"Particle Swarm Optimization Time: {pso_time:.4f} seconds\n")


Random vector: [0.77702869 0.28255589 0.37968447 0.41500396]

Counts before optimization: {'1': 115, '0': 909}
Energy: 0.6086008766691744
After optimization: {'1': 3, '0': 1021}
Success value: 0.994140625
Norm of the difference between partial traces: 0.05950138531954765
Simulated Annealing Time: 0.0952 seconds

Counts before optimization: {'1': 137, '0': 887}
Energy: 0.6086008766691744
After optimization: {'1': 56, '0': 968}
Success value: 0.890625
Norm of the difference between partial traces: 0.3699829277279632
Particle Swarm Optimization Time: 19.4537 seconds

